## 前置工作

In [65]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.signal

# 定义数据目录
directories = [
    "../data/library/txt/",
    "../data/library/csv/",
    "../data/test/single/txt/",
    "../data/test/single/csv/",
    "../data/test/multi/txt/",
    "../data/test/multi/csv/",
    "../data/library/txt_full/",
    "../data/library/csv_full/"
    ]

def create_library(directory):
    library = {}
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            mine_name = filename.split(".")[0]
            with open(os.path.join(directory, filename), 'r') as f:
                df = pd.read_csv(f)
                library[mine_name] = df
    return library

spectrum_lib = create_library(directories[1])
spectrum_lib2 = create_library(directories[7])
test_lib = create_library(directories[3])
multi_lib=create_library(directories[5])

## 预处理

In [66]:
def log_transform(spectrum, attr='reflectance'):
    y = spectrum[attr]
    y_new = np.log10(y)
    spectrum['log'] = y_new
    return y_new

def baseline_correction(spectrum, attr='log', window=61,order=3):
    y = spectrum[attr]
    y_base = scipy.signal.savgol_filter(y, window, order)
    y_new = spectrum['baseline'] =y-y_base
    return y_new

def normalize(spectrum, attr='log'):
    y = spectrum[attr]
    y_new = (y - y.min()) / (y.max() - y.min())
    spectrum['normalized'] = y_new
    return y_new

def denoise(spectrum, attr='log'):
    y = spectrum[attr]
    y_new = scipy.signal.wiener(y, mysize=5)
    spectrum['denoised'] = y_new
    return y_new



In [67]:
def preprocess(spectrum, bcwin=61, order=3):
    log_transform(spectrum)
    baseline_correction(spectrum,window=bcwin,order=order)
    normalize(spectrum, 'baseline')
    return spectrum

def preprocess_library(spectrum_lib, bcwin=61, order=3):
    for mine_name, df in spectrum_lib.items():
        preprocess(df,bcwin,order)

preprocess_library(spectrum_lib)
single_lib=test_lib
preprocess_library(single_lib)
preprocess_library(multi_lib)
preprocess_library(spectrum_lib2)

## 解混

In [68]:
def get_matrix(spectrum_lib, test_spectrum, attr='normalized'):
    # 从字典中取出所有的光谱数据
    normalized_arrays = [df[attr].values for df in spectrum_lib.values()]
    # 将所有的光谱数据合并成一个矩阵，每一行代表一个光谱数据
    merged_array = np.vstack(normalized_arrays)
    # 将矩阵转置
    transposed_array = merged_array.T
    # 取出测试光谱数据
    test_array = test_spectrum[attr].values
    return transposed_array, test_array

### Lasso

In [69]:
from sklearn.linear_model import Lasso
import numpy as np
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
from sklearn.model_selection import cross_val_score

def unmix_lasso(spectrum_lib,unknown_spectrum,attr='log',alpha=0.003,max_mines=2,min_account=0.1):
    # 获取端元数据和未知光谱数据
    X_known,y_unknown=get_matrix(spectrum_lib, unknown_spectrum,attr)
    X_known = np.array(X_known).T
    # 创建LASSO回归模型，并拟合已知光谱端元数据
    lasso = Lasso(alpha=alpha, positive=True)
    lasso.fit(X_known.T, y_unknown)
    # 获取端元系数（即端元的权重）
    endmember_coefficients = lasso.coef_
    # 处理端元系数
    mines=spectrum_lib.keys()
    # 提取权重大于0的端元
    endmember_coefficients_less = [(mine_name, coef) for mine_name, coef in zip(mines, endmember_coefficients) if coef > 0]
    # 对权重进行排序
    endmember_coefficients_less.sort(key=lambda x: x[1], reverse=True)
    # 只保留前max_mines个端元
    endmember_coefficients_less = endmember_coefficients_less[:max_mines]
    # 归一化权重，使其成为比例 for reporting
    total = sum([coef for mine_name, coef in endmember_coefficients_less])
    endmember_coefficients_less = [(mine_name, coef/total) for mine_name, coef in endmember_coefficients_less]
    # 剔除权重小于min_account的端元
    endmember_coefficients_less = [(mine_name, coef) for mine_name, coef in endmember_coefficients_less if coef > min_account]
    # 归一化权重，使其成为比例 for reporting
    total = sum([coef for mine_name, coef in endmember_coefficients_less])
    endmember_coefficients_less = [(mine_name, coef/total) for mine_name, coef in endmember_coefficients_less]
    # 创建一个只包含已选择端元的矩阵
    selected_endmember_matrix = np.column_stack([spectrum_lib[mine_name][attr].values for mine_name, coef in endmember_coefficients_less])
    # 使用这个矩阵和端元系数进行点积运算，得到拟合的混合光谱
    fitted_mixture_spectrum = np.dot(selected_endmember_matrix, [coef for mine_name, coef in endmember_coefficients_less])
    # 计算各种评价指标
    mse = mean_squared_error(y_unknown, fitted_mixture_spectrum)
    explained_var = explained_variance_score(y_unknown, fitted_mixture_spectrum)
    r2 = r2_score(y_unknown, fitted_mixture_spectrum)
    cross_val_scores = cross_val_score(lasso, X_known.T, y_unknown, cv=10)
    cross_val_scores_mean=np.mean(cross_val_scores)
    # 计算rss
    rss = np.sum((y_unknown - fitted_mixture_spectrum) ** 2)
    
    return endmember_coefficients_less,mse,explained_var,r2,cross_val_scores_mean,rss

## 岭回归

In [70]:
from sklearn.linear_model import Ridge
def unmix_ridge(spectrum_lib, unknown_spectrum, attr='log', alpha=0.2, max_mines=2,min_account=0.1):
    # 获取端元数据和未知光谱数据
    X_known, y_unknown = get_matrix(spectrum_lib, unknown_spectrum, attr)
    X_known = np.array(X_known).T
    # 创建Ridge回归模型，并拟合已知光谱端元数据
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_known.T, y_unknown)
    # 获取端元系数（即端元的权重）
    endmember_coefficients = ridge.coef_
    # 处理端元系数
    mines = spectrum_lib.keys()
    # 提取权重大于0的端元
    endmember_coefficients_less = [(mine_name, coef) for mine_name, coef in zip(mines, endmember_coefficients) if coef > 0]
    # 对权重进行排序
    endmember_coefficients_less.sort(key=lambda x: x[1], reverse=True)
    # 只保留前max_mines个端元
    endmember_coefficients_less = endmember_coefficients_less[:max_mines]
    # 归一化权重，使其成为比例 for reporting
    total = sum([coef for mine_name, coef in endmember_coefficients_less])
    endmember_coefficients_less = [(mine_name, coef/total) for mine_name, coef in endmember_coefficients_less]
    # 剔除权重小于min_account的端元
    endmember_coefficients_less = [(mine_name, coef) for mine_name, coef in endmember_coefficients_less if coef > min_account]
    # 归一化权重，使其成为比例 for reporting
    total = sum([coef for mine_name, coef in endmember_coefficients_less])
    endmember_coefficients_less = [(mine_name, coef/total) for mine_name, coef in endmember_coefficients_less]
    # 创建一个只包含已选择端元的矩阵
    selected_endmember_matrix = np.column_stack([spectrum_lib[mine_name][attr].values for mine_name, coef in endmember_coefficients_less])
    # 使用这个矩阵和端元系数进行点积运算，得到拟合的混合光谱
    fitted_mixture_spectrum = np.dot(selected_endmember_matrix, [coef for mine_name, coef in endmember_coefficients_less])
    # 计算各种评价指标
    mse = mean_squared_error(y_unknown, fitted_mixture_spectrum)
    explained_var = explained_variance_score(y_unknown, fitted_mixture_spectrum)
    r2 = r2_score(y_unknown, fitted_mixture_spectrum)
    cross_val_scores = cross_val_score(ridge, X_known.T, y_unknown, cv=10)
    cross_val_scores_mean = np.mean(cross_val_scores)
    # 计算rss
    rss = np.sum((y_unknown - fitted_mixture_spectrum) ** 2)
    return endmember_coefficients_less, mse, explained_var, r2, cross_val_scores_mean,rss

## 显示结果

In [71]:
import pandas as pd
from IPython.display import display

pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('display.max_rows', None)     # 显示所有行
pd.set_option('display.max_colwidth', None) # 显示列的完整内容

results = []
for mine_name, df in multi_lib.items():
    endmember_coefficients,mse,explained_var,r2,cross_val_scores_mean,rss = unmix_lasso(spectrum_lib, df,attr='normalized',alpha=0.001)
    endmember_coefficients_r,mse,explained_var,r2,cross_val_scores_mean,rss = unmix_ridge(spectrum_lib, df,attr='normalized',alpha=0.3)
    # results.append([mine_name, endmember_coefficients, mse, explained_var, r2, cross_val_scores_mean,rss])
    results.append([mine_name, endmember_coefficients,endmember_coefficients_r])

# df_results = pd.DataFrame(results, columns=['Mine Name', 'Endmember Coefficients', 'MSE', 'Explained Variance', 'R²', 'Cross-Validation Score', 'RSS'])
df_results = pd.DataFrame(results, columns=['Mine Name', 'Lasso','Ridge'])
display(df_results)

,Mine Name,Lasso,Ridge
0,al10ka90,"[(KAOLINOR, 0.6091534623444075), (HALLOYSI, 0.3908465376555925)]","[(HALLOYSI, 0.5772990311812448), (KAOLINOR, 0.42270096881875524)]"
1,al20ka80,"[(KAOLINOR, 0.6282933721577884), (HALLOYSI, 0.3717066278422116)]","[(HALLOYSI, 0.5493949542066923), (KAOLINOR, 0.4506050457933078)]"
2,al40ka60,"[(KAOLINOR, 0.5383284187757259), (HALLOYSI, 0.46167158122427415)]","[(HALLOYSI, 0.6142335433482294), (ALUNITE2, 0.3857664566517706)]"
3,alkaxx,"[(ALUNITE2, 0.6805837319621785), (HALLOYSI, 0.31941626803782164)]","[(ALUNITE2, 0.573913305607519), (DICKITE, 0.426086694392481)]"
4,alpyro2,"[(PYROPHYL, 0.871614388680507), (ALUNITE2, 0.12838561131949305)]","[(PYROPHYL, 0.7935990002370289), (HALLOYSI, 0.2064009997629711)]"
5,chris2a,"[(PYROPHYL, 0.6507836523209488), (ALUNITE2, 0.34921634767905113)]","[(PYROPHYL, 0.6140587333893656), (WATER2, 0.38594126661063444)]"
6,chris3b,"[(ALUNITE2, 0.5372664702120729), (KAOLINOR, 0.4627335297879271)]","[(ALUNITE2, 0.5162391589684439), (KAOLINOR, 0.48376084103155625)]"
7,chris4b,"[(NONTRON2, 0.7192024468035866), (SCHEELIT, 0.28079755319641353)]","[(NONTRON2, 0.6034024342463609), (NONTRON1, 0.39659756575363925)]"
8,chris5a,"[(ALBITE, 0.5654921730651755), (ILLITE, 0.43450782693482454)]","[(ALBITE, 0.544807123000458), (SODALITE, 0.4551928769995421)]"
9,chris6a,"[(HALLOYSI, 0.7431812306329536), (ANDESINE, 0.25681876936704634)]","[(HALLOYSI, 0.7307538705291293), (KAOLINOR, 0.2692461294708707)]"
